# Use ONNX model converted from CatBoost with `ibm-watsonx-ai`

This notebook facilitates `ONNX`, `CatBoost`, and `watsonx.ai Runtime` service. It contains steps and code to work with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in PyPI repository in order to convert the model to ONNX format. It also introduces commands for getting model and training data, persisting the model, deploying model, and scoring it.

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Learning goals

The learning goals of this notebook are:

-  Create a CatBoost model.
-  Convert CatBoost model to ONNX format.
-  Persist the converted model in the watsonx.ai Runtime repository.
-  Deploy model for online scoring using the client library.
-  Score sample records using the client library.


## Contents

This notebook contains the following parts:

1.	[Setting up the environment](#setup)
2.	[Creating a CatBoost model](#create)
3.	[Converting the CatBoost model to ONNX format](#convert)
4.	[Persisting the converted ONNX model](#upload)
5.	[Deploying and scoring the ONNX model](#deploy)
6.  [Cleaning up](#cleanup)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setting up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [watsonx.ai Runtime](https://cloud.ibm.com/catalog/services/watsonxai-runtime) instance (information on service plans and further reading can be found [here](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp)).

### 1.1. Installing and importing the `ibm-watsonx-ai` and dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install catboost==1.2.7 | tail -n 1
!pip install onnxruntime==1.16.3 | tail -n 1
!pip install onnx==1.16.2 | tail -n 1
!pip install scikit-learn==1.3 | tail -n 1

In [1]:
import getpass
import json
import onnx
import onnxruntime as ort
from catboost import CatBoostRegressor
from ibm_watsonx_ai import APIClient, Credentials

### 1.2. Connecting to watsonx.ai Runtime

Authenticate with the watsonx.ai Runtime service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

Get the value of `api_key` from the output.


Location of your watsonx.ai Runtime instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance INSTANCE_NAME
```

Get the value of `location` from the output.

**Tip**: You can generate your `Cloud API key` by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service-specific url by going to the [**Endpoint URLs** section of the watsonx.ai Runtime docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance details.

You can also get the service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key, and paste it below.

**Action**: Enter your `api_key` and `location` in the following cells.

In [2]:
api_key = getpass.getpass("Please enter your api key (hit enter): ")

Please enter your api key (hit enter):  ········


In [3]:
location = 'ENTER YOUR LOCATION HERE'

If you are running this notebook on Cloud, you can access the `location` via:

```
location = os.environ.get("RUNTIME_ENV_REGION")
```

In [4]:
credentials = Credentials(
    api_key=api_key,
    url=f'https://{location}.ml.cloud.ibm.com'
)

In [5]:
client = APIClient(credentials)

### 1.3. Working with spaces

First of all, you need to create a space that will be used for your work. If you do not already have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select the watsonx.ai Runtime instance and press Create
- Copy the `space_id` and paste it below

**Tip**: You can also use the `ibm_watsonx_ai` SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [6]:
space_id = 'ENTER YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai Runtime, you need to set **space** which you will be using.

In [8]:
client.set.default_space(space_id)

'SUCCESS'

<a id="create"></a>
## 2. Creating a CatBoost model
In this section, you will create and train a sample CatBoost model.

### 2.1. Creating and training the regressor model

In [9]:
train_data = [[1, 4, 5, 6],
              [4, 5, 6, 7],
              [30, 40, 50, 60]]

eval_data = [[2, 4, 6, 8],
             [1, 4, 50, 60]]

train_labels = [10, 20, 30]

model = CatBoostRegressor(iterations=2, learning_rate=1, depth=2)
model.fit(train_data, train_labels)

0:	learn: 6.1237244	total: 63.2ms	remaining: 63.2ms
1:	learn: 4.5927933	total: 63.4ms	remaining: 0us


<a id="evaluation"></a>
### 2.2. Evaluating the model

In [10]:
model.predict(eval_data)

array([15.625, 18.125])

<a id="convert"></a>
## 3. Converting the CatBoost model to ONNX format
In this section, you will convert the model to the ONNX format.

### 3.1. Converting the model

In [11]:
onnx_model_name = "catboost_model.onnx"

model.save_model(
    onnx_model_name,
    format="onnx",
    export_parameters={
        'onnx_domain': 'ai.catboost',
        'onnx_model_version': 1,
        'onnx_doc_string': 'test model for Regressor',
        'onnx_graph_name': 'CatBoostModel_for_Regression'
    }
)

**Note:** To convert a pipeline with a CatBoost model refer to the [documentation](https://onnx.ai/sklearn-onnx/auto_tutorial/plot_gexternal_catboost.html).

<a id="evaluation-onnx"></a>
### 3.2. Evaluating the ONNX Model

After exporting the model, you should verify its integrity and ensure that it functions as expected. We will use `onnxruntime` to load the model and perform inference on the test data. Additionally, we’ll use `onnx`'s `checker` module to validate the exported ONNX model.


In [12]:
onnx_model = onnx.load(onnx_model_name)
onnx.checker.check_model(onnx_model)

In [13]:
session = ort.InferenceSession(onnx_model_name)
input_data = {session.get_inputs()[0].name: eval_data}
session.run([], input_data)

2025-01-29 22:08:44.964782 [W:onnxruntime:, execution_frame.cc:857 VerifyOutputSizes] Expected shape from model of {-1} does not match actual shape of {2,1} for output predictions


[array([[15.625],
        [18.125]], dtype=float32)]

As you can see, the predicted values are consistent with those calculated in the [evaluation](#evaluation) section.

<a id="upload"></a>
## 4. Persisting the converted ONNX model

In this section, you will learn how to store your converted ONNX model in watsonx.ai Runtime repository using the watsonx.ai Runtime SDK.

### 4.1. Publishing the model in the watsonx.ai Runtime repository

Define model name, type and software spec.

In [14]:
sofware_spec_id = client.software_specifications.get_id_by_name("onnxruntime_opset_19")
onnx_model_zip = "catboost_onnx.zip"

In [15]:
!zip {onnx_model_zip} {onnx_model_name}

  adding: catboost_model.onnx (deflated 61%)


In [16]:
metadata = {
    client.repository.ModelMetaNames.NAME: 'CatBoost to ONNX converted model',
    client.repository.ModelMetaNames.TYPE: 'onnxruntime_1.16',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sofware_spec_id
}

published_model = client.repository.store_model(
    model=onnx_model_zip,
    meta_props=metadata
)

### 4.2. Getting model details

In [ ]:
published_model_id = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_id)
print(json.dumps(model_details, indent=2))

<a id="deploy"></a>
## 5. Deploying and scoring the ONNX model

In this section you'll learn how to create an online scoring service and predict on unseen data.

<a id="create_deploy"></a>
### 5.1. Creating an online deployment for the published model

In [18]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of CatBoost to ONNX converted model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: 'c7e324a9-d95a-42b4-934d-9f4df5358856' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
.
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='057bc87b-7067-4957-813e-700ddf0c958f'
-----------------------------------------------------------------------------------------------




In [19]:
deployment_id = client.deployments.get_id(created_deployment)

Now you can print an online scoring endpoint. 

In [ ]:
client.deployments.get_scoring_href(created_deployment)

<a id="get_deploy"></a>
### 5.2. Getting deployment details

In [ ]:
client.deployments.get_details(deployment_id)

<a id="score"></a>
### 5.3. Scoring

You can use this method to perform a test scoring request against the deployed model.

Prepare scoring payload with records to score.

In [22]:
scoring_payload = {
    "input_data": [
        {
            "values": eval_data

        }
    ]
}

Use the `client.deployments.score()` method to run scoring.

In [23]:
predictions = client.deployments.score(deployment_id, scoring_payload)

Let's print the result of predictions.

In [24]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "id": "predictions",
      "values": [
        [
          15.625
        ],
        [
          18.125
        ]
      ]
    }
  ]
}


As you can see, the predicted values are consistent with those calculated in the [evaluation](#evaluation-onnx) section.

<a id="cleanup"></a>
## 6. Cleaning up

If you want to clean up after the notebook execution, i.e. remove any created assets like:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 7. Summary and next steps

 You successfully completed this notebook! You learned how to use ONNX, Catboost machine learning library as well as watsonx.ai Runtime for model creation and deployment. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=wx)_ for more samples, tutorials, documentation, how-tos, and blog posts.

Copyright © 2025 IBM. This notebook and its source code are released under the terms of the MIT License.